In [7]:
import pandas as pd
import geopandas as gpd
import json
import plotly.graph_objects as go
import numpy as np
from urllib.request import urlopen

# Load GeoJSON file
with open("merged_provinces.geojson") as f:
    data = json.load(f)

# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame.from_features(data["features"])

# Read the CSV
df = pd.read_csv('iraq.csv')

/var/folders/1w/6vqfb5jj0nv8pf9fqf_nwwfh0000gn/T/ipykernel_13746/2146968529.py:16: DtypeWarning:

Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.



In [2]:
gdf["NAME_1"].unique

<bound method Series.unique of 0          MNF-W
1          MND-C
2         MND-SE
3    MND-BAGHDAD
4          MND-N
Name: NAME_1, dtype: object>

In [3]:
df["Region"].unique()

array(['MNF-W', 'MND-N', 'MND-C', 'MND-SE', 'MND-BAGHDAD'], dtype=object)

In [8]:
# Convert 'Datetime' column to datetime and extract the month
df['Datetime'] = pd.to_datetime(df['Datetime'])
df['Month'] = df['Datetime'].dt.to_period('M')

# Group by Region and Month and sum the KIA columns
df_grouped = df.groupby(['Region', 'Month'])[['Enemy_KIA', 'Friend_KIA', 'Civilian_KIA', 'Host_nation_KIA']].sum().reset_index()

# Merge with the geo dataframe
merged = gdf.merge(df_grouped, left_on='NAME_1', right_on='Region', how='inner')

In [9]:
merged.dtypes

geometry            geometry
NAME_1                object
Region                object
Month              period[M]
Enemy_KIA              int64
Friend_KIA             int64
Civilian_KIA           int64
Host_nation_KIA        int64
dtype: object

## the current only working solution for plot 2 which play button and slider dont work anymore

In [10]:
import pandas as pd
import json
import plotly.graph_objects as go
import numpy as np

# Load GeoJSON file
with open("merged_provinces.geojson") as f:
    data = json.load(f)

# Read the CSV
df = pd.read_csv('iraq.csv')

# Convert 'Datetime' column to datetime and extract the month
df['Datetime'] = pd.to_datetime(df['Datetime'])
df['Month'] = df['Datetime'].dt.to_period('M')

# Group by Region and Month and sum the KIA columns
df_grouped = df.groupby(['Region', 'Month'])[['Enemy_KIA', 'Friend_KIA', 'Civilian_KIA', 'Host_nation_KIA']].sum().reset_index()

# Calculate the sum of the four columns
df_grouped['Total_KIA'] = df_grouped[['Enemy_KIA', 'Friend_KIA', 'Civilian_KIA', 'Host_nation_KIA']].sum(axis=1)

# Calculate the percentage of 'Enemy_KIA'
df_grouped['Enemy_KIA_Percentage'] = df_grouped['Enemy_KIA'] / df_grouped['Total_KIA'] * 100
df_grouped['Friend_KIA_Percentage'] = df_grouped['Friend_KIA'] / df_grouped['Total_KIA'] * 100
df_grouped['Civilian_KIA_Percentage'] = df_grouped['Civilian_KIA'] / df_grouped['Total_KIA'] * 100
df_grouped['Host_nation_KIA_Percentage'] = df_grouped['Host_nation_KIA'] / df_grouped['Total_KIA'] * 100

# Merge with the geojson data
# Assuming 'NAME_1' in geojson data is corresponding to 'Region' in your CSV
for feature in data['features']:
    feature['id'] = feature['properties']['NAME_1']

# Convert 'Month' to string for animation_frame
df_grouped['Month'] = df_grouped['Month'].astype(str)

months = np.sort(df_grouped.Month.unique())

fig = go.Figure()

for kia_type in ['Enemy_KIA_Percentage', 'Friend_KIA_Percentage', 'Civilian_KIA_Percentage', 'Host_nation_KIA_Percentage']:
    fig_frames = []
    for month in months:
        month_df = df_grouped[df_grouped.Month == month]
        fig_frames.append(go.Frame(data=[go.Choroplethmapbox(geojson=data, 
                                                             locations=month_df.Region, 
                                                             z=month_df[kia_type], 
                                                             colorscale="YlOrRd",
                                                             zmin=0,
                                                             zmax=100,
                                                             marker_opacity=0.9, 
                                                             marker_line_width=0)]))
    fig.add_trace(go.Choroplethmapbox(visible=(kia_type == 'Enemy_KIA_Percentage'),
                                      geojson=data, 
                                      locations=df_grouped[df_grouped.Month == months[0]].Region, 
                                      z=df_grouped[df_grouped.Month == months[0]][kia_type], 
                                      colorscale="YlOrRd",
                                      zmin=0,
                                      zmax=100,
                                      marker_opacity=0.9, 
                                      marker_line_width=0,
                                      name=kia_type))

fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            showactive=False,
            buttons=list([
                dict(label="Play",
                     method="animate",
                     args=[None, {"frame": {"duration": 500, "redraw": True},
                                  "fromcurrent": True,
                                  "transition": {"duration": 300,
                                                 "easing": "quadratic-in-out"}}])
            ]),
        ),
        dict(
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.1,
            yanchor="top",
            buttons=list([
                dict(label="Enemy_KIA_Percentage",
                     method="restyle",
                     args=[{"z": [df_grouped.Enemy_KIA_Percentage]}]),
                dict(label="Friend_KIA_Percentage",
                     method="restyle",
                     args=[{"z": [df_grouped.Friend_KIA_Percentage]}]),
                dict(label="Civilian_KIA_Percentage",
                     method="restyle",
                     args=[{"z": [df_grouped.Civilian_KIA_Percentage]}]),
                dict(label="Host_nation_KIA_Percentage",
                     method="restyle",
                     args=[{"z": [df_grouped.Host_nation_KIA_Percentage]}])
            ]),
        )
    ],
    sliders=[dict(steps = [dict(method= 'animate',
                                args= [[f'frame_{k}'], 
                                {"frame": {"duration": 300, "redraw": True},
                                 "mode": "immediate",
                                 "transition": {"duration": 300}}],
                                label=month) for k, month in enumerate(months)],
                active=0,
                transition= {"duration": 300 },
                x=0,
                y=0, 
                currentvalue=dict(font=dict(size=12), 
                                  prefix='Month: ', 
                                  visible=True, 
                                  xanchor='center'))],
    mapbox_style="carto-positron",
    mapbox_zoom=3,
    mapbox_center = {"lat": 33.3152, "lon": 44.3661},  # approximate center of Iraq
    title="Killed in Action by Region"
)

fig.show()



/var/folders/1w/6vqfb5jj0nv8pf9fqf_nwwfh0000gn/T/ipykernel_13746/488663861.py:11: DtypeWarning:

Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.



In [8]:

# Define a function to create a map for a particular month
def create_map_for_month(year_month):
    filtered = merged[merged['Month'] == year_month].reindex()
    merged['Month'] = merged['Month'].astype(str)
    print(filtered.head(10))
    fig = px.choropleth_mapbox(merged, geojson=merged.geometry, locations=merged.index,
                               color='Enemy_KIA',  # replace with the column you want to visualize
                               mapbox_style="carto-positron",
                               color_continuous_scale="ylorrd",
                               labels={'NAME_1':'Coalition Force'},  # update labels as per your data
                               zoom=5, center = {"lat": filtered.geometry.centroid.y.mean(), "lon": filtered.geometry.centroid.x.mean()},
                               animation_frame='Month'  # add time slider
                              )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

# Create a map for January 2004
create_map_for_month(pd.Period('2005-06'))

Empty GeoDataFrame
Columns: [geometry, NAME_1, Region, Month, Enemy_KIA, Friend_KIA, Civilian_KIA, Host_nation_KIA]
Index: []


KeyboardInterrupt: 

In [29]:

# Define a function to create a map for a particular month
def create_map_for_month(year_month):
    filtered = merged[merged['Month'] == year_month].reindex()
    print(filtered.head(10))
    fig = px.choropleth_mapbox(filtered, geojson=filtered.geometry, locations=filtered.index,
                               color='Enemy_KIA',  # replace with the column you want to visualize
                               mapbox_style="carto-positron",
                               color_continuous_scale="ylorrd",
                               labels={'NAME_1':'Coalition Force'},  # update labels as per your data
                               zoom=5, center = {"lat": filtered.geometry.centroid.y.mean(), "lon": filtered.geometry.centroid.x.mean()},
                              )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

# Create a map for January 2004
create_map_for_month(pd.Period('2005-06'))

Empty GeoDataFrame
Columns: [geometry, NAME_1, Region, Month, Enemy_KIA, Friend_KIA, Civilian_KIA, Host_nation_KIA]
Index: []


NameError: name 'px' is not defined